# Procesamiento de grafos en Spark con GraphFrames

Recordemos que GraphFrames no es parte de Spark. Es un paquete externo que permite procesar grafos usando Spark DataFrames en lugar de los RDD menos cómodos, como hacía el antiguo módulo GraphX que sí es parte de Spark. Se espera que GraphFrames se incorpore oficialmente a Spark en el futuro cercano.

Mientras tanto, primero debemos instalar el paquete GraphFrames para python, que es solo un wrapper de Python para el código Scala que contiene la implementación realmente paralela y distribuida. Podemos instalar el paquete de Python a través de `pip3 install`. Para la implementación de Scala, tenemos que indicar el paquete GraphFrames como "Spark package" que queremos usar en la opción de configuración `--packages` antes de crear el objeto SparkSession. Esto descargará la implementación GraphFrames para Scala sobre la marcha. El paquete de Python se basa por completo en dicho código Scala y generará un error si no se encuentra. Para que los notebooks de JupyterLab ya lleven esto incorporado, se añadió la propiedad de `spark.jars.packages` al crear el cluster en Dataproc. 

### Descargamos el fichero csv de vuelos y lo subimos a HDFS

In [ ]:
!wget https://raw.githubusercontent.com/olbapjose/xapi-clojure/master/flights_jan08.csv
!hdfs dfs -copyFromLocal flights_jan08.csv /tmp

### Descargamos el paquete graphframes de python con el comando pip habitual de Python

In [ ]:
!pip install graphframes-dev

In [ ]:
from pyspark.sql import functions as F
from graphframes import GraphFrame

flightsDF = spark.read.option("header", "true")\
                      .option("inferSchema", "true")\
                      .csv("/tmp/flights_jan08.csv")

verticesDF = flightsDF.select(F.col("Origin").alias("id")).distinct().cache()

edgesDF = flightsDF.withColumnRenamed("Origin", "src")\
                   .withColumnRenamed("Dest", "dst")\
                   .select("src", "dst", "Distance")\
                   .distinct()\
                   .cache() # select a few columns just to keep things simple
                
graph = GraphFrame(verticesDF, edgesDF)

Vamos a calcular los aeropuertos con más vuelos como aquellos con el grado más alto en su vértice (suma de ambos).
Es sorprendente que el aeropuerto LAS (Las Vegas - McCarran International Airport) quede en primer lugar en número de vuelos!

In [ ]:
graph.degrees.orderBy(F.col("degree").desc()).show(3)

In [ ]:
graph.inDegrees.orderBy(F.col("inDegree").desc()).show(3)

In [ ]:
graph.outDegrees.orderBy(F.col("outDegree").desc()).show(3)

## PageRank para determinar la importancia global de los vértices

Si ejecutamos PageRank, que es un poco más sofisticado que contar simplemente el número de aristas que llegan o salen de cada vértice, confirma que LAS tiene importancia estratégica en nuestro grafo en base a las conexiones con otros aeropuertos.

In [ ]:
ranks = graph.pageRank(resetProbability=0.15, maxIter=10)

### ADVERTENCIA: pageRank tarda unos minutos en ejecutarse. Descomentar la siguiente línea si realmente queremos ejecutarlo :-)

In [7]:
#ranks = graph.pageRank(resetProbability=0.15, maxIter=10)
ranks.vertices.orderBy(F.col("pagerank").desc()).show()

+---+------------------+
| id|          pagerank|
+---+------------------+
|LAS|3.9681005880868927|
|MDW| 3.464188596739582|
|PHX|2.9996296419306288|
|BWI| 2.859044345021574|
|MCO| 2.506806330762528|
|TPA| 2.325980728922274|
|ABQ|  2.28062606472766|
|HOU| 2.242821989285586|
|BNA|2.1393487543945984|
|SAN|  2.10337109753917|
|MCI|1.9976511399324188|
|AUS|1.8642893476111473|
|STL|1.8263822546604833|
|ONT|1.7722329347353263|
|SAT|1.5303551318597992|
|OAK|1.4886721798643956|
|SMF|1.4779258223049847|
|LAX|1.4134227909403223|
|SLC| 1.362280151671023|
|JAX|1.2637766842227056|
+---+------------------+
only showing top 20 rows



## Componentes conexas

En nuestor grafo es posible llegar a cualquier aeropuerto desde cualquier otro, y por tanto solo hay una componente conexa y la columna `component` (identificador de la componente) tiene el mismo valor para todos los vértices.

In [ ]:
spark.sparkContext.setCheckpointDir("/home/")
conCompResult = graph.connectedComponents(checkpointInterval=10)
conCompResult.show(100)

## Consultas por estructura: rutas de vuelo entre aeropuertos sin conexión directa

Estamos indicando que queremos encontrar vértices **a**, **b** y **c** para que haya una arista de **a** a **b**, otra de **b** a **c**, pero no de **a** a **c**, por lo tanto, **a** y **c** no están conectados en un solo salto sino que requieren al menos dos. La restricción adicional evita que **a** y **c** sean el mismo vértice, ya que ningún aeropuerto está conectado consigo mismo, por lo que cada aeropuerto cumpliría individualmente la condición respecto a sí mismo.

In [ ]:
res = graph\
 .find("(a)-[]->(b); (b)-[]->(c); !(a)-[]->(c)")\
 .filter("c.id !=a.id")

res.show()

## Caminos más cortos: Breadth-first search

Vamos a encontrar el camino más corto entre dos aeropuertos que no están directamente conectados

<div class="alert alert-danger">
    <b>IMPORTANTE</b>: BFS (Breadth-first search) en Spark calcula el camino mínimo en términos de <b>número de saltos</b> entre dos vértices. No tiene en cuenta el peso de las aristas. Se podría implementar pero de forma personalizada, no con la función bfs().
</div>

Como hay varios caminos que tienen 2 saltos entre ABQ y BNA, la función `bfs` devuelve todos ellos.

In [ ]:
paths = graph.bfs(fromExpr = "id = 'ABQ'", toExpr= "id = 'BNA'")
paths.show()